In [1]:
!pip install openai

In [3]:
import json
from openai import OpenAI

client = OpenAI(api_key="KEY")




SCHEMA = {
    "type": "object",
    "properties": {
        "label": {"type": "string", "enum": ["related", "unrelated", "uncertain"]},
        "confidence": {"type": "number", "minimum": 0, "maximum": 1},
        "rationale": {"type": "string"}
    },
    "required": ["label", "confidence", "rationale"],
    "additionalProperties": False
}

INSTRUCTIONS = """Classify Serbian news headlines as related/unrelated to the anti-corruption protests and student blockades.
Use only the headline. If unclear, use uncertain.
Keep rationale short (<= 15 words)."""

def label_headline(headline: str) -> dict:
    resp = client.responses.create(
        model="gpt-5.2",
        instructions=INSTRUCTIONS,
        input=[{"role": "user", "content": headline}],
        text={
            "format": {
                "type": "json_schema",
                "name": "headline_label",
                "strict": True,
                "schema": SCHEMA,
            }
        },
        temperature=0.0,
    )
    return json.loads(resp.output_text)





{'label': 'related', 'confidence': 0.93, 'rationale': 'Pominje studente i blokadu mosta u Beogradu, tipično za proteste.'}


In [5]:
import pandas as pd

n1=pd.read_csv("/content/n1_lemmatized.csv")
headline=n1["headline"]

In [17]:
gpt_result = pd.DataFrame()

In [19]:


for h in headline.iloc[100:1000]:
    row = pd.Series(label_headline(h)).to_frame().T
    gpt_result = pd.concat([gpt_result, row], ignore_index=True)

gpt_result = gpt_result[["label", "confidence", "rationale"]]



In [21]:
gpt_result.to_csv("gpt_result.csv")